In [217]:
#librerias
import pandas as pd


In [218]:
#Lectura de datos de archivo
datos_brutos = pd.read_csv("Building_Permits.csv")
#datos_brutos.columns.values

c:\program files\python39\lib\site-packages\IPython\core\interactiveshell.py:3012: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# LIMPIEZA DE DATOS

In [219]:
#Funcion formato de fecha
def formato_fecha(df,columna_afectada):
     for i in range(len(list(df[columna_afectada]))):
        aux=str(df.at[i, columna_afectada])
        df.at[i, columna_afectada]=str(aux[6:]+'-'+aux[0:2]+'-'+aux[3:5]+' 00:00:00')
#Funcion formato de fecha
def formato_fecha_null(df,columna_afectada):
     for i in range(len(list(df[columna_afectada]))):
        aux=str(df.at[i, columna_afectada])
        if aux=='nan':
            df.at[i, columna_afectada]='NULL'
        else:
            df.at[i, columna_afectada]=str(aux[6:]+'-'+aux[0:2]+'-'+aux[3:5]+' 00:00:00')
#Funcion separacion de numero y sufijo numero de calle
def separacion_ultima_letra(df,columna_afectada,columna_letra):
    for i in range(len(list(df[columna_afectada]))):
        aux=str(df.at[i, columna_afectada])
        if str(aux).isdigit():
            df.at[i, columna_afectada]=int(aux)
        else:
            df.at[i, columna_afectada]=int(aux[:len(aux)-1])
            df.at[i, columna_letra]=aux[-1:]
#Relleno de datos con datos y tipo de dato
def Relleno_data(df,columnas,relleno,dataType):
    for i in columnas:
        df[i]=df[i].fillna(value=relleno)
        df[i]=df[i].astype(dataType)
        if dataType=='str':
            df[i]=df[i].str.upper()
            
#Datos string a mayusculas
def String_mayus(df,columnas):
    for i in columnas:
        df[i]=df[i].astype('str')
        df[i]=df[i].str.upper()

#Transformacion de fecha
def fecha_formato(df,columnas):
    for i in columnas:
        formato_fecha(datos_limpios,i)
        String_mayus(datos_limpios,[i])
        
def fecha_formato_null(df,columnas):
    for i in columnas:
        formato_fecha_null(datos_limpios,i)
        String_mayus(datos_limpios,[i])

def Relleno_int(df,columnas):
    for i in columnas:
        df[i]=df[i].astype('int64')
        
#Datos int con NULL
def Relleno_int_null(df,columnas):
    Relleno_data(df,columnas,-1,'int64')
    String_mayus(df,columnas)
    for i in columnas:
        df.loc[df[i]=='-1',i]='NULL'
        df[i]=df[i].astype('str')
        df[i]=df[i].str.upper()
#Funcion para rellenar 2 campos int
def relleno_2int(df,columna_afectada,columna_llenado):
    for i in range(len(list(df[columna_afectada]))):
        aux=str(df.at[i, columna_afectada])
        aux1=str(df.at[i, columna_llenado])
        if str(aux).isdigit() and aux1=='nan':
            df.at[i, columna_llenado]=int(aux)
    for i in range(len(list(df[columna_afectada]))):
        aux1=str(df.at[i, columna_afectada])
        aux=str(df.at[i, columna_llenado])
        if str(aux).isdigit() and aux1=='nan':
            df.at[i, columna_afectada]=int(aux)
    Relleno_data(df,[columna_afectada,columna_llenado],0,'int64')
    
#Verificacion y limpieza de datos
#print(datos_brutos.dtypes)
datos_limpios=datos_brutos.copy()

relleno_string=['Street Number Suffix','Unit Suffix','Description',
                'Neighborhoods - Analysis Boundaries','Location']
relleno_int=['Permit Type','Record ID']
relleno_int0=['Unit','Proposed Units', 'Plansets','Existing Units','Existing Construction Type',
              'Proposed Construction Type']
relleno_int_null=[ 'Estimated Cost','Revised Cost','Supervisor District','Zipcode']
relleno_string_mayus=['Permit Number','Permit Type Definition','Street Name','Current Status']
relleno_fechas=['Permit Creation Date','Current Status Date','Filed Date']
relleno_fechas_null=['Issued Date','Completed Date', 'First Construction Document Date','Permit Expiration Date']
relleno_N=['Structural Notification','Voluntary Soft-Story Retrofit','Fire Only Permit','TIDF Compliance','Site Permit']

columna='Block'
separacion_ultima_letra(datos_limpios,columna,'Street Number Suffix')
datos_limpios[columna]=datos_limpios[columna].astype('int64')

columna='Lot'
separacion_ultima_letra(datos_limpios,columna,'Street Number Suffix')
datos_limpios[columna]=datos_limpios[columna].astype('int64')



#Relleno strin mayus
String_mayus(datos_limpios,relleno_string_mayus)
#Relleno strin null, str
Relleno_data(datos_limpios,relleno_string,'NULL','str')
#Relleno int0l, int64
Relleno_data(datos_limpios,relleno_int0,0,'int64')
#Relleno string null, ST
Relleno_data(datos_limpios,['Street Suffix'],'ST','str')
#Relleno de fechas
fecha_formato(datos_limpios,relleno_fechas)
fecha_formato_null(datos_limpios,relleno_fechas_null)
#Rellenos de int64
Relleno_int(datos_limpios,relleno_int)
#Rellenos de int64 con mull
Relleno_int_null(datos_limpios,relleno_int_null)
#Relleno string null, N
Relleno_data(datos_limpios,relleno_N,'N','str')
#Relleno string null, NO USING
Relleno_data(datos_limpios,['Existing Use'],'NO USING','str')
#Relleno string null,GENERAL
Relleno_data(datos_limpios,['Proposed Use'],'GENERAL','str')
#Relleno de number stories
relleno_2int(datos_limpios,'Number of Existing Stories','Number of Proposed Stories')
#Relleno string null,constr type 0
Relleno_data(datos_limpios,['Existing Construction Type Description',
                            'Proposed Construction Type Description'],'constr type 0','str')

#Generacion de archivo csv con datos limpios
datos_limpios.to_csv('clean_Building_Permits.csv', index=False)
datos_limpios






,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,SIGN - ERECT,2015-05-06 00:00:00,326,23,140,NULL,ELLIS,ST,...,3,CONSTR TYPE 3,0,CONSTR TYPE 0,N,3,TENDERLOIN,94102,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,SIGN - ERECT,2016-04-19 00:00:00,306,7,440,NULL,GEARY,ST,...,3,CONSTR TYPE 3,0,CONSTR TYPE 0,N,3,TENDERLOIN,94102,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,ADDITIONS ALTERATIONS OR REPAIRS,2016-05-27 00:00:00,595,203,1647,NULL,PACIFIC,AV,...,1,CONSTR TYPE 1,1,CONSTR TYPE 1,N,3,RUSSIAN HILL,94109,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,OTC ALTERATIONS PERMIT,2016-11-07 00:00:00,156,11,1230,NULL,PACIFIC,AV,...,5,WOOD FRAME (5),5,WOOD FRAME (5),N,3,NOB HILL,94109,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,DEMOLITIONS,2016-11-28 00:00:00,342,1,950,NULL,MARKET,ST,...,3,CONSTR TYPE 3,0,CONSTR TYPE 0,N,6,TENDERLOIN,94102,"(37.78315261897309, -122.40950883997789)",144548169992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,M862628,8,OTC ALTERATIONS PERMIT,2017-12-05 00:00:00,113,17,1228,A,MONTGOMERY,ST,...,0,CONSTR TYPE 0,0,CONSTR TYPE 0,N,NULL,NULL,NULL,NULL,1489337276729
198896,201712055595,8,OTC ALTERATIONS PERMIT,2017-12-05 00:00:00,271,14,580,NULL,BUSH,ST,...,5,WOOD FRAME (5),5,WOOD FRAME (5),N,NULL,NULL,NULL,NULL,1489462354993
198897,M863507,8,OTC ALTERATIONS PERMIT,2017-12-06 00:00:00,4318,19,1568,NULL,INDIANA,ST,...,0,CONSTR TYPE 0,0,CONSTR TYPE 0,N,NULL,NULL,NULL,NULL,1489539379952
198898,M863747,8,OTC ALTERATIONS PERMIT,2017-12-06 00:00:00,298,29,795,NULL,SUTTER,ST,...,0,CONSTR TYPE 0,0,CONSTR TYPE 0,N,NULL,NULL,NULL,NULL,1489608233656


# ETL DB

In [342]:
#importacion de librerias
import sqlite3 
import os 
import os.path 
import ctypes

databaseFile = 'DB.db' 
sqlFile = '.\\DB_Transaccional.sql'

# Delete the old table 
if os.path.isfile(databaseFile): 
    os.remove(databaseFile)
# Crear tabla
qry = open(sqlFile, 'r').read() 
sqlite3.complete_statement(qry) 
conn = sqlite3.connect('.\\'+databaseFile) 
cursor = conn.cursor() 
try: 
    cursor.executescript(qry) 
except Exception as e: 
    print('error')
cursor.close()
conn.close()

In [343]:
#librerias
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
engine = create_engine('sqlite:///'+databaseFile)
# SQL Expression Language crea metadatos que contienen objetos que definen la tabla de clientes
metadata = MetaData()
# Este método instancia las tablas que ya
metadata.create_all(engine)
# Revisando esto, podemos ver la estructura de la tabla y los tipos de variables 
inspector = inspect(engine)

#Funciones de carga
def updateData(name_table,data_db,name_columns): 
    print(name_columns)
    aux_values='?'
    aux_data=[]
    aux_columns=name_columns[0]
    aux_data.append(list(data_db[name_columns[0]]))
    entities=[]
    aux_entities=[]
    for i in range(len(name_columns)-1):
        aux_values+=',?'
        aux_columns+=','+name_columns[i+1]
        aux_data.append(list(data_db[name_columns[i+1]]))
    for i in range(len(aux_data[0])):
        #aux_entities.append(i)
        for j in range(len(aux_data)):
            aux_entities.append(aux_data[j][i])
        entities.append(aux_entities)
        aux_entities=[]
    with engine.connect() as con:
        for i in entities:
            con.execute('INSERT INTO '+name_table+'('+aux_columns+') VALUES('+aux_values+')', i)
    con.close()
#Ingreso de datos
datos={
    '[Permit Type]':list(datos_limpios['Permit Type']),
    '[Permit Type Definition]':list(datos_limpios['Permit Type Definition'])
}
df=pd.DataFrame(datos)
df=df.drop_duplicates(subset=['[Permit Type]'])
updateData('permitType',df,list(df.columns.values))
print(df)
datos={
    '[Unit]':list(datos_limpios['Unit']),
    '[Unit Suffix]':list(datos_limpios['Unit Suffix'])
}
df=pd.DataFrame(datos)
filtro = df['[Unit]'] !='NULL'
df = df[filtro]
df=df.drop_duplicates(subset=['[Unit]','[Unit Suffix]'])
updateData('unit',df,list(df.columns.values))
print(df)
datos={
    '[Existing Units]':list(datos_limpios['Existing Units']),
    '[Existing Use]':list(datos_limpios['Existing Use'])
}
df=pd.DataFrame(datos)
df=df.drop_duplicates(subset=['[Existing Units]'])
updateData('unitEx',df,list(df.columns.values))
print(df)
datos={
    '[Neighborhoods - Analysis Boundaries]':list(datos_limpios['Neighborhoods - Analysis Boundaries']),
    '[Supervisor District]':list(datos_limpios['Supervisor District']),
    '[Zipcode]':list(datos_limpios['Zipcode']),
    '[Location]':list(datos_limpios['Location'])
}
df=pd.DataFrame(datos)
df=df.drop_duplicates(subset=['[Neighborhoods - Analysis Boundaries]'])
updateData('neib',df,list(df.columns.values))
print(df)
datos={
    '[Street Name]':list(datos_limpios['Street Name']),
    '[Street Suffix]':list(datos_limpios['Street Suffix']),
    '[Street Number]':list(datos_limpios['Street Number']),
    '[Street Number Suffix]':list(datos_limpios['Street Number Suffix']),
    '[Lot]':list(datos_limpios['Lot']),
    '[Block]':list(datos_limpios['Block'])
}

df=pd.DataFrame(datos)
df=df.drop_duplicates(subset=['[Street Name]','[Street Suffix]','[Street Number]','[Street Number Suffix]',
                             '[Lot]','[Block]'])
aux1=df.copy()
updateData('pathTable',df,list(df.columns.values))
print(df)